In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [32]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

## Assignment 4

## Task 1 

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [33]:
np.set_printoptions(suppress=True) # easier to check values rn when we dont care too much about the exactness

In [49]:
# in minimization think probability of being wrong, in max probability of acceptance
# probability of sample A dominating sample B in max is P(A > B) is
def prob_selection(a, b, sigma_a, sigma_b):
    s = sigma_a / (sigma_b + 0.00001) # just to prevent division by zero
    m = (a - b) / sigma_b
    top = -2.5*m / np.sqrt(2 + 2*s**2)
    bot = 1 + np.exp(top)
    return 1 / bot


a = np.array([1,1,3])
b = np.array([2,5,1])
sigma_a = 1.0
sigma_b = 1.0

prob_selection(a, b, sigma_a, sigma_b)

array([0.22270014, 0.00669285, 0.92414182])

So we can see that probability of being wrong with first items (1 and 2) by choosing a is 0.22. Again for second items (1,5) thats 0.007.  And also with 3rd items (3,1) its 0.92. So we can clearly see its working properly.

## Task 2

Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)

- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [42]:
dtlz2_3 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

In [43]:
def create_samples(dec_dim, samples):
    # create samples 
    x = lhs(dec_dim, samples)     
    return np.asarray(x)

In [44]:
x = create_samples(5, 50)
print(x.shape)
eval_results = dtlz2_3.evaluate(x)
y = eval_results.objectives
print(y[:5])

(50, 5)
[[1.1893948  0.17827164 0.1959251 ]
 [0.05729974 0.11694473 1.06965455]
 [0.24042055 0.74171975 1.05247565]
 [0.88144829 0.930741   0.4431327 ]
 [0.92001799 0.49817906 1.11552057]]


In [45]:
print(y[:,0])

[1.1893948  0.05729974 0.24042055 0.88144829 0.92001799 0.17385373
 0.25250361 0.98912795 0.27973573 0.10875958 0.48843669 1.12852472
 0.18311602 0.53590127 0.71384297 0.8465182  0.22945687 0.01626112
 0.13166219 0.85366239 0.68474785 0.70350046 0.85339772 0.42743307
 0.21017408 0.03569362 0.11921928 1.05129454 1.15501347 0.21246462
 0.8594671  0.36990048 0.71579045 0.52058036 0.62377022 0.21384557
 0.94192856 1.60108798 0.07989734 0.44294343 0.57026266 0.00196047
 0.09424005 0.7217975  0.00187209 1.27111086 0.54936379 1.14372167
 0.00764793 1.22283513]


In [46]:
## build surrogates

kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 20.0), nu=1.5)
    
y1 = y[:,0]
gpr1 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y1)

y2 = y[:,1]
gpr2 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y2)

y3 = y[:,2]
gpr3 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y3)

In [47]:
## resampling

x_samples = create_samples(5, 20)

# predict
y1_sample, y1_std = gpr1.predict(x_samples, return_std=True) 

# predict
y2_sample, y2_std = gpr2.predict(x_samples, return_std=True) 

# predict
y3_sample, y3_std = gpr3.predict(x_samples, return_std=True) 

In [48]:
a = np.array([1,1])
b = np.array([2,0])

# eq 15: MO P(A < B) : product to k, P(Aj < Bj)
def smaller(a, b):
    return j

# eq 17: MO P(A > B) : product to k, 1 - P(Aj < Bj)
def bigger(a, b):
    return 1 - smaller(a, b)


# eq 18: MO P(A = B) : 1 - P(A < B) - P(A > B)
def equals(a, b):
    return 1 - smaller(a, b) - bigger(a, b)

# eq 22: proba ranking : sum to n, P(Fj > Fi) + 0.5 sum to n P(Fj = Fi) - 0.5
def probabilitistic_ranking():
    ranks = 0
    #non_dom = 
    
    return ranks

In [ ]:
a = np.array([1,1])
b = np.array([2,0])

smaller(a,b)

In [ ]:
# objective values from fitness measurements
f1 = np.array([0,1])
f2 = np.array([1,2])
# f1 dominates f2